# 🚀 Official OpenAI SDK in Python

Welcome to this notebook! In this notebook, we will be using the official OpenAI SDK in Python to interact with OpenAI's powerful language models. The SDK provides a simple and efficient way to access the capabilities of OpenAI's models, enabling us to perform a variety of tasks such as text generation, completion, and more.

## 📦 Installation

First, let's install the OpenAI SDK. You can do this using pip.

## 📝 Usage

Once the SDK is installed, you can start using it in your Python code. Set your OpenAI API key and make requests to the OpenAI API to get responses from the language models.

## 📚 Documentation

For more detailed information on how to use the OpenAI SDK, please refer to the official documentation.

Happy coding! 🎉

In [ ]:
%pip install openai==1.58.1
%pip install python-dotenv==1.0.1
%pip install pandas==2.2.3

### Load all environment variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

chat_deployment_name = os.getenv("CHAT_DEPLOYMENT_NAME")
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")
embedding_model=os.getenv("EMBEDDING_MODEL")


In [ ]:
from openai import AzureOpenAI  

client = AzureOpenAI(api_key=api_key, azure_endpoint=base_url, api_version="2024-08-01-preview")

chat_history = [
        {
            "role": "system", 
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Tell me 5 funny fact about Quebec City."
        }
]

completion = client.chat.completions.create(
    model=chat_deployment_name,
    messages=chat_history
)

print(completion.to_json())

# Add message to chat history
chat_history.append({
    "role": completion.choices[0].message.role,
    "content": completion.choices[0].message.content
})

In [ ]:
print(completion.choices[0].message.content)

Print all the histories and responses from the chatbot.

In [ ]:
import json

print(json.dumps(chat_history, indent=4))

In [ ]:
chat_history.append({
    "role": "user",
    "content": "Give me more details related to the first fact ?"
})

completion = client.chat.completions.create(
    model=chat_deployment_name,
    messages=chat_history
)

print(completion.choices[0].message.content)

# Embedding with OpenAI 🌟

Embedding is a technique used to represent data in a continuous vector space. OpenAI leverages embeddings to enhance various applications, especially in natural language processing.

## What is Embedding? 🤔

Embedding transforms high-dimensional data into a lower-dimensional space, making it easier to process and analyze.

## Why Use Embedding? 🛠️

1. **Dimensionality Reduction**: Simplifies data while preserving essential information.
2. **Improved Performance**: Enhances model efficiency and accuracy.
3. **Semantic Meaning**: Captures relationships between data points.

## OpenAI and Embedding 🚀

OpenAI uses embeddings to:
- **Understand Text**: Represent words and sentences in a vector space.
- **Improve Models**: Enhance the performance of language models.
- **Analyze Data**: Reveal hidden patterns and relationships.

## Conclusion 🎯

Embedding is a powerful tool in OpenAI's arsenal, enabling efficient data processing and improved model performance.

In [6]:
response = client.embeddings.create(input="This is an amazing cake, I love chocolate cake they are the best", model=embedding_model)

In [ ]:
print(response.to_json())

# 🌟 Structured Outputs in JSON 🌟

JSON is one of the most widely used formats for data exchange in applications.

## 🚀 Benefits of Structured Outputs

- **Reliable type-safety**: Ensures correctly formatted responses without validation or retries.
- **Explicit refusals**: Detect safety-based model refusals programmatically.
- **Simpler prompting**: Achieve consistent formatting without strongly worded prompts.

In [8]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model=chat_deployment_name,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

print(event)

name='Science Fair Attendance' date='Friday (specific date not provided)' participants=['Alice', 'Bob']


With the Structured Outputs you can provide the chain of thoughts and responses in a structured way.

In [ ]:
class Step(BaseModel):
    explanation: str
    output: str

class AnswerReasoning(BaseModel):
    steps: list[Step]
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model=chat_deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Guide the user through the answer step by step."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."}
    ],
    response_format=AnswerReasoning,
)    

answer_reasoning = completion.choices[0].message.parsed

print(answer_reasoning.model_dump_json(indent=4))

# 🌟 Introduction to Function Calling with OpenAI Models

Function calling enables developers to connect language models to external data and systems. You can define a set of functions as tools that the model has access to, and it can use them when appropriate based on the conversation history. You can then execute those functions on the application side, and provide results back to the model.

In [ ]:
# Create a method that will return the weather in major region in Middle-Earth
def get_weather(region: str) -> str:
    weather_data = {
        "Shire": "Sunny",
        "Mordor": "Hot and dry",
        "Rivendell": "Mild and rainy",
        "Gondor": "Warm and breezy",
        "Rohan": "Windy",
        "Mirkwood": "Foggy and damp",
        "Isengard": "Stormy",
        "Lothlorien": "Pleasant and cool"
    }
    
   # Normalize the region name to lower case for comparison
    region = region.lower()
    
    for key in weather_data:
        if key.lower() in region:
            return weather_data[key]
    
    return "Unknown region"

print(get_weather("Shire"))  # Output: Sunny
print(get_weather("Mordor"))  # Output: Hot and dry
print(get_weather("Unknown"))  # Output: Unknown region

In [26]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "get_weather",
          "strict": True,
          "description": "Get the weather in a region in Middle-Earth. Call this whenever you need to know the weather in middle-earth, for example when an user asks 'What is the weather today in the Shire?'",
          "parameters": {
              "type": "object",
              "properties": {
                  "region": {
                      "type": "string",
                      "description": "The region of middle-earth.",
                  },
              },
              "required": ["region"],
              "additionalProperties": False,
          },
      }
  }
]

chat_history =[]

messages = [
  {
      "role": "system",
      "content": "You are a helpful support assistant. Use the supplied tools to assist the user."
  },
  {
      "role": "user",
      "content": "Hi, what is the weather today in the Mirkwood?"
  }
]

chat_history.extend(messages)

response = client.chat.completions.create(
  model=chat_deployment_name,
  messages=messages,
  tools=tools,
)

In [27]:
# Show if a function should be called
print(response.to_json(indent=4))

{
    "id": "chatcmpl-ApMTXCsfIb57BYAONGLbeqWdtPxnr",
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": "call_ajvLEgcDjZfWF3H5GEfLK38q",
                        "function": {
                            "arguments": "{\"region\":\"Mirkwood\"}",
                            "name": "get_weather"
                        },
                        "type": "function"
                    }
                ]
            },
            "content_filter_results": {}
        }
    ],
    "created": 1736804487,
    "model": "gpt-4o-2024-11-20",
    "object": "chat.completion",
    "system_fingerprint": "fp_82ce25c0d4",
    "usage": {
        "completion_tokens": 16,
        "prompt_tokens": 115,
        "total_tokens

In [30]:
# Call the function

tool_call = response.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["region"])

print(result)


Foggy and damp


In [31]:
messages.append(response.choices[0].message)  # append model's tool call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": result
})

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(completion_2.choices[0].message.content)

The weather today in Mirkwood is foggy and damp.
